In [2]:
import sys
print (sys.version)

2.7.12 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:42:40) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


In [108]:
import numpy as np
import os
import pandas as pd
import sys
import matplotlib.pyplot as plt
import sklearn.cross_validation as skcv
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.preprocessing import Imputer
from sklearn.metrics import accuracy_score

%matplotlib inline

In [44]:
fcData = pd.read_csv('./abide_fc_data_fisher_z.csv')
fcData['func_perc_quart'] = pd.qcut(fcData['func_perc_fd'], q=4, labels=False)
fcData.head()

,Unnamed: 0,subject,func_perc_fd,AGE_YRS,SEX,DX_GROUP,#2001_#2002,#2001_#2101,#2001_#2102,#2001_#2111,...,#9130_#9150,#9130_#9160,#9130_#9170,#9140_#9150,#9140_#9160,#9140_#9170,#9150_#9160,#9150_#9170,#9160_#9170,func_perc_quart
0,1,50003,67.164179,24.0,1,1,1.373414,0.409453,0.789834,0.455512,...,0.940071,0.798582,0.471726,0.857586,0.650342,0.592167,0.843476,0.599385,0.875041,3
1,2,50004,14.427861,19.0,1,1,0.824020,0.481031,0.258819,0.246055,...,0.334438,0.234090,0.130916,0.632359,0.421152,-0.029851,0.866840,-0.061741,0.027180,2
2,3,50005,10.945274,13.0,2,1,1.135987,1.116633,0.944885,0.942790,...,0.799417,0.778156,0.370868,0.590871,0.558449,0.262422,0.525498,0.256752,0.203965,2
3,4,50006,1.492537,13.0,1,1,1.456934,0.634692,0.303313,0.166852,...,0.420481,0.061504,0.391978,0.459101,0.095503,0.320199,0.121727,0.198423,0.114551,1
4,5,50007,18.905473,17.0,1,1,0.682998,0.959670,0.642405,0.977074,...,0.934691,0.486346,0.626130,0.886814,0.432660,0.455022,0.656619,0.483390,0.405080,3


In [113]:
n_iter=10
age_range=(6,18)
motion_threshold=5
sample_size=100

In [114]:
fcData_threshed = fcData.query("AGE_YRS >= " + str(age_range[0]) + 
                               " & AGE_YRS <= " + str(age_range[1]) + 
                               " & func_perc_fd <= " + str(motion_threshold))
strata = fcData_threshed.groupby(['DX_GROUP', 'func_perc_quart', 'SEX']).grouper.group_info[0]
labels = fcData_threshed['DX_GROUP'].as_matrix()
features = fcData_threshed.loc[:,'#2001_#2002':'#9160_#9170']
features_imputed = Imputer(missing_values='NaN', strategy='mean', axis=0).fit_transform(features)

In [115]:
penalty = 'l1'
algorithm = LinearSVC(penalty = penalty)
algorithm.set_params(dual=False)
paramsToSearch = []        
paramsToSearch.append({'C': [.001,.005,.01,.1,1,10]})
clf = GridSearchCV(algorithm, 
                   paramsToSearch, 
                   cv=StratifiedShuffleSplit(strata, n_iter, test_size=int(sample_size*.3)),
                   n_jobs=4)
clf.fit(features_imputed, labels)

/home/jflournoy/miniconda3/envs/kidsPy2/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/jflournoy/miniconda3/envs/kidsPy2/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/jflournoy/miniconda3/envs/kidsPy2/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
arr

GridSearchCV(cv=StratifiedShuffleSplit(labels=[2 2 ..., 4 4], n_iter=10, test_size=30, random_state=None),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid=[{'C': [0.001, 0.005, 0.01, 0.1, 1, 10]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [116]:
#Get training accuracy
trainingPredictions = clf.predict(features_imputed)
accuracy = accuracy_score(trainingPredictions, labels)
print "Training accuracy: %f" % accuracy

Training accuracy: 0.920330
